In [1]:
import warnings
warnings.filterwarnings('ignore')
from numpy import *
import numpy as np
import pandas as pd
import sys, os
import torch
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

from GCFNN import GCFNN
import torch.nn.functional as F
from numpy.random import randint

In [2]:
def concatenated_tensor(X_set, M):
    concatenated_tensor_list = []
    # 遍历索引范围为 0 到 M-1 的值
    for i in range(M.shape[1]):
        # 拼接 X_set[i]
        concatenated_tensor_list.append(X_set[i])
        ## 将 adj_list[i] 转换为密集张量并拼接
        #adj_dense_tensor = adj_list[i].to_dense()
        #concatenated_tensor_list.append(adj_dense_tensor)
        
    # 将 tr_M 转换为张量并拼接
    M_tensor = torch.from_numpy(M)
    concatenated_tensor_list.append(M_tensor)
    # 沿着 dim=1 拼接所有张量
    concatenated_tensor = torch.cat(concatenated_tensor_list, dim=1)
    return concatenated_tensor

In [3]:
# read data
meta_data=pd.read_csv('meta.csv',index_col=0)
metabolomics=pd.read_csv('metabolomics.csv',index_col=0).fillna(0)
metagenomics=pd.read_csv('metagenomics.csv',index_col=0)
data_class={'nonIBD':0,'CD':2,'UC':1}
meta_data['diagnosis']=meta_data['diagnosis'].map(data_class)

X1=metagenomics
X2=metabolomics
print('输入各组学数据表：',X1.shape,X2.shape,meta_data.shape)

#重构数据表和标签表
index_new = X1.index&X2.index&meta_data[meta_data['diagnosis'] != 2].index
                                        
meta_data=meta_data.reindex(index_new)
X1=array(X1.reindex(index_new))
X2=array(X2.reindex(index_new))
print('集合所有样本重构数据表：',X1.shape,X2.shape,meta_data.shape)
X_set=[X1,X2]
M        = len(X_set)
Mask     = np.ones([np.shape(X_set[0])[0], M])
#生成表示组学缺失情况的掩码矩阵，将缺失组学部分填补为0
for m_idx in range(M):
    Mask[np.isnan(X_set[m_idx]).all(axis=1), m_idx] = 0
    #X_set[m_idx][Mask[:, m_idx] == 0] = np.mean(X_set[m_idx][Mask[:, m_idx] == 1], axis=0)
    X_set[m_idx][Mask[:, m_idx] == 0] = 0
    
#对宏基因组数据预处理
def log(base,x):
    return np.log(x)/np.log(base)
X_set[0]=log(2,2*X_set[0]+0.00001)

#print(Mask)
#print(X_set)
Y = meta_data['diagnosis'].tolist()
Y = np.array(Y)
Y_onehot = np.zeros((Y.shape[0], 2))
Y_onehot[np.arange(Y.shape[0]), Y] = 1
print('标签转换为one-hot编码：', Y_onehot.shape, np.sum(Y_onehot[:, 0]))

输入各组学数据表： (1638, 578) (546, 596) (2891, 1)
集合所有样本重构数据表： (207, 578) (207, 596) (207, 1)
标签转换为one-hot编码： (207, 2) 105.0


In [4]:
# 指定保存模型的文件夹路径
folder_path = 'model_shap'
# 创建文件夹
os.makedirs(folder_path, exist_ok=True)

cv = StratifiedKFold(n_splits=5,shuffle = True,random_state = 50)
auc_cv = []
acc_cv = []
f1_cv = []
for i,(train_index, test_index) in enumerate(cv.split(X_set[0],np.argmax(Y_onehot,axis=1))):
    tr_X_set, te_X_set, va_X_set = {}, {}, {}
    for m in range(len(X_set)):
        tr_X_set[m],tr_Y_onehot,tr_M = X_set[m][train_index],Y_onehot[train_index],Mask[train_index]
        te_X_set[m],te_Y_onehot,te_M = X_set[m][test_index],Y_onehot[test_index],Mask[test_index]
        #归一化
        #print(te_Y_onehot)
    def Normalize(data):
        """
        :param data:Input data
        :return:normalized data
        """
        mean = np.mean(data)
        mx = np.max(data)
        mn = np.min(data)
        return mean, mx, mn
    
    for m in range(M):
        mean, mx, mn = Normalize(tr_X_set[m])
        tr_X_set[m] = (tr_X_set[m] - mean) / (mx - mn)
        te_X_set[m] = (te_X_set[m] - mean) / (mx - mn)
    
    x_dim_set    = [tr_X_set[m].shape[1] for m in range(len(tr_X_set))]
    y_dim        = np.shape(tr_Y_onehot)[1]
    
    input_dims = {
        'x_dim_set': x_dim_set,
        'y_dim': y_dim,
        'z_dim': 50,
        #'steps_per_batch': steps_per_batch
    }
    network_settings = {
        'dim_enc': 100,
        'num_layers_enc': 3,   #view-specific
        'dim_specificpre': 100,
        'num_layers_specificpre': 2,  #multi-view
        'dim_joint_pre': 100,
        'num_layers_jointpre': 2,
        'dropout': 0.5,
        'edge_per_node': 8, 
        'ITERATION': 2000
    }
    
    for m in range(M):
        tr_X_set[m] = torch.from_numpy(tr_X_set[m])
        te_X_set[m] = torch.from_numpy(te_X_set[m])
    tr_Y_onehot = torch.from_numpy(tr_Y_onehot)
    te_Y_onehot = torch.from_numpy(te_Y_onehot)
    #tansform
    tr_input = concatenated_tensor(tr_X_set, tr_M)
    te_input = concatenated_tensor(te_X_set, te_M)
    GPU_use = True
    model = GCFNN(input_dims, network_settings, cuda = GPU_use)
    if GPU_use: 
        model = model.cuda()
        tr_input = tr_input.cuda()
        te_input = te_input.cuda()
        tr_Y_onehot = tr_Y_onehot.cuda()
        te_Y_onehot = te_Y_onehot.cuda()
    model_save, maxf1, maxacc, maxauc = model.train_model(train_input = tr_input, test_input = te_input, alpha = 1.0, beta = 0.01, l_rate = 0.0005, tr_Y_onehot = tr_Y_onehot, te_Y_onehot = te_Y_onehot)
    model_path = os.path.join(folder_path, 'model_'+str(i)+'.pth')
    torch.save(model_save, model_path)
    f1_cv.append(maxf1)
    acc_cv.append(maxacc)
    auc_cv.append(maxauc)
print('f1',f1_cv)
print('average maxf1',sum(f1_cv)/5)
print('acc',acc_cv)
print('average maxacc',sum(acc_cv)/5)
print('auc',auc_cv)
print('average maxauc',sum(auc_cv)/5)

Train F1: 0.7702
00050: TRAIN| LT=1.875 LP=0.556 LKL=11.821 LPS=1.107 LKLS=9.390 | 
Test F1: 0.8500
Test ACC: 0.8571
Test AUC: 0.9773
Train F1: 0.9481
00100: TRAIN| LT=1.341 LP=0.243 LKL=12.541 LPS=0.844 LKLS=12.829 | 
Test F1: 0.9268
Test ACC: 0.9286
Test AUC: 0.9932
Train F1: 0.9441
00150: TRAIN| LT=1.135 LP=0.134 LKL=13.142 LPS=0.731 LKLS=13.898 | 
Test F1: 0.9524
Test ACC: 0.9524
Test AUC: 0.9932
Train F1: 0.9697
00200: TRAIN| LT=1.055 LP=0.106 LKL=13.137 LPS=0.698 LKLS=11.977 | 
Test F1: 0.8889
Test ACC: 0.8810
Test AUC: 0.9887
Train F1: 0.9689
00250: TRAIN| LT=0.968 LP=0.085 LKL=13.315 LPS=0.623 LKLS=12.624 | 
Test F1: 0.9524
Test ACC: 0.9524
Test AUC: 0.9932
Train F1: 0.9811
00300: TRAIN| LT=0.955 LP=0.089 LKL=13.549 LPS=0.597 LKLS=13.286 | 
Test F1: 0.9756
Test ACC: 0.9762
Test AUC: 0.9955
Train F1: 0.9686
00350: TRAIN| LT=0.943 LP=0.065 LKL=13.661 LPS=0.610 LKLS=13.089 | 
Test F1: 0.9302
Test ACC: 0.9286
Test AUC: 0.9955
Train F1: 0.9814
00400: TRAIN| LT=0.847 LP=0.084 LKL=13.